In [18]:
import pandas as pd
import numpy as np

In [19]:
atlasData = pd.read_pickle('../../ATLAS_data/CutFlow_Chargino.pcl')
for c in atlasData.columns:
    if c in ['mLLP','mLSP','tau_ns','Total MC Events','VertexEff Strategy','width', 'Total xsec (pb)']:
        continue
    atlasData[c] = np.array(atlasData[c].to_list())[:,1]
atlasData['label'] = 'ATLAS'

atlasData.T

,0
mLLP,1300.0
tau_ns,10
Total,1.0
Event and jet Cleaning,0.995
Trigger,0.539
Primary vertex,0.539
$E_{T}^{miss}>170$ GeV,0.381
Track in PV,0.378
$p_{T} > 50$ GeV,0.326
Track quality requirements,0.279


In [20]:
recastData = pd.read_pickle('./charginoCutFlow.pcl')
recastData = recastData[(recastData['mLLP'] == 1300.) & (np.round(recastData['tau_ns']) == 10.0)]
for c in recastData.columns:
    if c in ['mLLP','mLSP','tau_ns','Total MC Events','VertexEff Strategy','width', 'Total xsec (pb)']:
        continue
    
    recastData[c] = np.array(recastData[c].to_list())[:,0]

renameCols = {'Event Sel.' : '$E_{T}^{miss}>170$ GeV'}
recastData['label'] = 'Recast'
recastData.rename(columns=renameCols,inplace=True)
recastData.reset_index(drop=True, inplace=True)
recastData.T

,0
mLLP,1300.0
mLSP,1299.0
tau_ns,10.0
width,0.0
Total MC Events,10752
Total xsec (pb),0.00025
Total,1.0
$n_{Charged} > 0$,1.0
$n_{Charged} > 0$ (+mu tag),0.9612
Trigger,0.5571


In [21]:
df = atlasData.merge(recastData,how='outer')
df.drop(columns=[c for c in df.columns if c not in atlasData.columns],inplace=True)
df.T


/tmp/ipykernel_37944/4067106348.py:1: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  df = atlasData.merge(recastData,how='outer')


,0,1
mLLP,1300.0,1300.0
tau_ns,10.0,10.0
Total,1.0,1.0
Event and jet Cleaning,0.995,NaN
Trigger,0.539,0.5571
Primary vertex,0.539,NaN
$E_{T}^{miss}>170$ GeV,0.381,0.3834
Track in PV,0.378,NaN
$p_{T} > 50$ GeV,0.326,NaN
Track quality requirements,0.279,NaN


In [22]:
combData = atlasData.T.copy()
for index in atlasData.T.index.values:
    atlasVals = atlasData.T.loc[index]
    if index == 'label':
        combData.loc[index] = ['Recast (ATLAS)']*len(atlasVals)
        continue
    if index in ['mLLP','mLSP','tau_ns']:
        continue
    if isinstance(atlasVals[0],str):
        continue
    if index in recastData.columns:
        recastVals = recastData.T.loc[index]
    else:
        recastVals = [np.nan]*len(atlasVals)
    merged = ['%1.3f (%1.3f)' %x for x in zip(recastVals,atlasVals)]
    merged = [x.replace('nan', '-----') for  x in merged[:]]
    # print(index)
    # print(merged)
    combData.loc[index] = merged

In [23]:
combData

,0
mLLP,1300.0
tau_ns,10
Total,1.000 (1.000)
Event and jet Cleaning,----- (0.995)
Trigger,0.557 (0.539)
Primary vertex,----- (0.539)
$E_{T}^{miss}>170$ GeV,0.383 (0.381)
Track in PV,----- (0.378)
$p_{T} > 50$ GeV,----- (0.326)
Track quality requirements,----- (0.279)


In [24]:
print(combData.to_markdown())

|                                                | 0              |
|:-----------------------------------------------|:---------------|
| mLLP                                           | 1300.0         |
| tau_ns                                         | 10             |
| Total                                          | 1.000 (1.000)  |
| Event and jet Cleaning                         | ----- (0.995)  |
| Trigger                                        | 0.557 (0.539)  |
| Primary vertex                                 | ----- (0.539)  |
| $E_{T}^{miss}>170$ GeV                         | 0.383 (0.381)  |
| Track in PV                                    | ----- (0.378)  |
| $p_{T} > 50$ GeV                               | ----- (0.326)  |
| Track quality requirements                     | ----- (0.279)  |
| Track isolation                                | ----- (0.244)  |
| $p_{T} > 120$ GeV                              | 0.264 (0.242)  |
| Track momentum uncertainty                    